# **Name - Shradha Pujari**
# **Data Science Intern at LetsGrowMore**
# **Advanced Level Task 2- Next Word Prediction** 




In [ ]:
import tensorflow as tf
import string
import requests

In [ ]:
response=requests.get("http://www.gutenberg.org/cache/epub/5200/pg5200.txt")

In [ ]:
response.text[:1500]


'\ufeffThe Project Gutenberg EBook of Metamorphosis, by Franz Kafka\r\nTranslated by David Wyllie.\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **\r\n**     Please follow the copyright guidelines in this file.     **\r\n\r\n\r\nTitle: Metamorphosis\r\n\r\nAuthor: Franz Kafka\r\n\r\nTranslator: David Wyllie\r\n\r\nRelease Date: August 16, 2005 [EBook #5200]\r\nFirst posted: May 13, 2002\r\nLast updated: May 20, 2012\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK METAMORPHOSIS ***\r\n\r\n\r\n\r\n\r\nCopyright (C) 2002 David Wyllie.\r\n\r\n\r\n\r\n\r\n\r\n  Metamorphosis\r\n  Franz Kafka\r\n\r\nTranslated by David Wyllie\r\n\r\n\r\n\r\nI\r\n\r\n\r\nOne morning, when Gregor

In [ ]:
data = response.text.split('\n')
data[0]


'\ufeffThe Project Gutenberg EBook of Metamorphosis, by Franz Kafka\r'

In [ ]:
data = data[253:]
data[0]


'away from the bed, bend down with the load and then be patient and\r'

In [ ]:
len(data)


2110

In [ ]:
data = " ".join(data)
data[:1000]


'away from the bed, bend down with the load and then be patient and\r careful as he swang over onto the floor, where, hopefully, the\r little legs would find a use.  Should he really call for help\r though, even apart from the fact that all the doors were locked?\r Despite all the difficulty he was in, he could not suppress a smile\r at this thought.\r \r After a while he had already moved so far across that it would have\r been hard for him to keep his balance if he rocked too hard.  The\r time was now ten past seven and he would have to make a final\r decision very soon.  Then there was a ring at the door of the flat.\r "That\'ll be someone from work", he said to himself, and froze very\r still, although his little legs only became all the more lively as\r they danced around.  For a moment everything remained quiet.\r "They\'re not opening the door", Gregor said to himself, caught in\r some nonsensical hope.  But then of course, the maid\'s firm steps\r went to the door as ever and o

In [ ]:
def clean_text(doc):
 tokens = doc.split()
 table = str.maketrans('', '', string.punctuation)
 tokens = [w.translate(table) for w in tokens]
 tokens = [word for word in tokens if word.isalpha()]
 tokens = [word.lower() for word in tokens]
 return tokens
tokens = clean_text(data)
print(tokens[:50])

['away', 'from', 'the', 'bed', 'bend', 'down', 'with', 'the', 'load', 'and', 'then', 'be', 'patient', 'and', 'careful', 'as', 'he', 'swang', 'over', 'onto', 'the', 'floor', 'where', 'hopefully', 'the', 'little', 'legs', 'would', 'find', 'a', 'use', 'should', 'he', 'really', 'call', 'for', 'help', 'though', 'even', 'apart', 'from', 'the', 'fact', 'that', 'all', 'the', 'doors', 'were', 'locked', 'despite']


In [ ]:
len(tokens)

22607

In [ ]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
   break

print(len(lines))


22556


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]


array([ 103,   29,    1,  245, 2883,   98,   14,    1, 1435,    3,   48,
         30,  618,    3,  756,   13,    6, 1434,  107,  165,    1,  149,
         86, 2880,    1,   78,  225,   21,  530,   12,  156,  193,    6,
        142,  754,   17,  180,  116,   49, 1433,   29,    1,  753,   11,
         26,    1,  455,   58,  617,  329])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1


In [ ]:
y = to_categorical(y, num_classes=vocab_size)


In [ ]:
seq_length = X.shape[1]
seq_length

50

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))


In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            144250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 2885)              291385    
Total params: 586,535
Trainable params: 586,535
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
model.fit(X, y, batch_size = 256, epochs = 100)

Epoch 1/100
89/89 [==============================] - 36s 403ms/step - loss: 6.6534 - accuracy: 0.0464
Epoch 2/100
89/89 [==============================] - 33s 370ms/step - loss: 6.1880 - accuracy: 0.0540
Epoch 3/100
89/89 [==============================] - 32s 365ms/step - loss: 6.1551 - accuracy: 0.0540
Epoch 4/100
89/89 [==============================] - 33s 365ms/step - loss: 6.0518 - accuracy: 0.0540
Epoch 5/100
89/89 [==============================] - 33s 368ms/step - loss: 5.9690 - accuracy: 0.0541
Epoch 6/100
89/89 [==============================] - 33s 367ms/step - loss: 5.8392 - accuracy: 0.0593
Epoch 7/100
89/89 [==============================] - 32s 359ms/step - loss: 5.7397 - accuracy: 0.0664
Epoch 8/100
89/89 [==============================] - 33s 365ms/step - loss: 5.6532 - accuracy: 0.0732
Epoch 9/100
89/89 [==============================] - 34s 377ms/step - loss: 5.5703 - accuracy: 0.0771
Epoch 10/100
89/89 [==============================] - 35s 391ms/step - loss: 5.497

89/89 [==============================] - 19s 213ms/step - loss: 2.7815 - accuracy: 0.3491
Epoch 81/100
89/89 [==============================] - 19s 212ms/step - loss: 2.7616 - accuracy: 0.3514
Epoch 82/100
89/89 [==============================] - 20s 227ms/step - loss: 2.7357 - accuracy: 0.3542
Epoch 83/100
89/89 [==============================] - 22s 246ms/step - loss: 2.7153 - accuracy: 0.3560
Epoch 84/100
89/89 [==============================] - 19s 214ms/step - loss: 2.6952 - accuracy: 0.3615
Epoch 85/100
89/89 [==============================] - 19s 219ms/step - loss: 2.6790 - accuracy: 0.3665
Epoch 86/100
89/89 [==============================] - 18s 207ms/step - loss: 2.6547 - accuracy: 0.3705
Epoch 87/100
89/89 [==============================] - 18s 205ms/step - loss: 2.6396 - accuracy: 0.3710
Epoch 88/100
89/89 [==============================] - 19s 212ms/step - loss: 2.6334 - accuracy: 0.3736
Epoch 89/100
89/89 [==============================] - 19s 212ms/step - loss: 2.6175 - 

In [ ]:
seed_text=lines[12343]
seed_text

'condition seemed serious enough to remind even his father that gregor despite his current sad and revolting form was a family member who could not be treated as an enemy on the contrary as a family there was a duty to swallow any revulsion for him and to be patient just'

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
        
        y_predict = model.predict_classes(encoded)
    
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break 
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)


In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

'to be patient easy back closer that would threatened that he would have to be heard home the kitchen locked himself hidden as he was not a loss but the room he isnt we cant see it out of the door and listening him and the chief clerk but was still a minor discourtesy and the rooms gentleman in the endless holiday i appear immediate notice with the same conclusion for the time being blossoming as all playing he had poured their door on him and then they answered answered with confirmation of him and were very times to transport'

# **Thank You!**